# ResNet For CIFAR0-10

In [27]:
import torch
from torchvision.models import resnet18

import torchvision
import torchvision.transforms as transforms

import os
from torchsummary import summary

### Model Construction

Here we use torch build of ResNet18

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = resnet18().to(device)

In [28]:
summary(model, (3, 32, 232))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 16, 116]           9,408
       BatchNorm2d-2          [-1, 64, 16, 116]             128
              ReLU-3          [-1, 64, 16, 116]               0
         MaxPool2d-4            [-1, 64, 8, 58]               0
            Conv2d-5            [-1, 64, 8, 58]          36,864
       BatchNorm2d-6            [-1, 64, 8, 58]             128
              ReLU-7            [-1, 64, 8, 58]               0
            Conv2d-8            [-1, 64, 8, 58]          36,864
       BatchNorm2d-9            [-1, 64, 8, 58]             128
             ReLU-10            [-1, 64, 8, 58]               0
       BasicBlock-11            [-1, 64, 8, 58]               0
           Conv2d-12            [-1, 64, 8, 58]          36,864
      BatchNorm2d-13            [-1, 64, 8, 58]             128
             ReLU-14            [-1, 64

Construction Complete

### Data Preprocessing

In [16]:

transform_train = transforms.Compose([
    #transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


In [17]:
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train) # change transform in future
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test) # change transform in future
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1,
                       weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


In [14]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()



In [24]:
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc
    return acc


In [23]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()


Epoch: 0
Saving..

Epoch: 1

Epoch: 2

Epoch: 3

Epoch: 4
Saving..

Epoch: 5
Saving..

Epoch: 6

Epoch: 7

Epoch: 8
Saving..

Epoch: 9

Epoch: 10

Epoch: 11

Epoch: 12

Epoch: 13

Epoch: 14

Epoch: 15

Epoch: 16

Epoch: 17

Epoch: 18

Epoch: 19

Epoch: 20

Epoch: 21

Epoch: 22

Epoch: 23

Epoch: 24

Epoch: 25

Epoch: 26

Epoch: 27

Epoch: 28

Epoch: 29

Epoch: 30

Epoch: 31

Epoch: 32

Epoch: 33

Epoch: 34

Epoch: 35

Epoch: 36

Epoch: 37

Epoch: 38

Epoch: 39

Epoch: 40

Epoch: 41

Epoch: 42

Epoch: 43

Epoch: 44

Epoch: 45

Epoch: 46

Epoch: 47

Epoch: 48

Epoch: 49

Epoch: 50

Epoch: 51

Epoch: 52

Epoch: 53

Epoch: 54

Epoch: 55

Epoch: 56

Epoch: 57

Epoch: 58

Epoch: 59

Epoch: 60

Epoch: 61

Epoch: 62

Epoch: 63

Epoch: 64

Epoch: 65

Epoch: 66

Epoch: 67

Epoch: 68

Epoch: 69

Epoch: 70

Epoch: 71

Epoch: 72

Epoch: 73

Epoch: 74

Epoch: 75

Epoch: 76

Epoch: 77

Epoch: 78

Epoch: 79

Epoch: 80

Epoch: 81

Epoch: 82

Epoch: 83

Epoch: 84

Epoch: 85

Epoch: 86

Epoch: 87

Epoch

In [26]:
print('Accuracy: ',test(0))

Accuracy:  64.41
